## DIM0549 - Grafos : 1º Unidade, Grupo 5

### Classes abstratas contendo metodos basicos das classes *Vertice* e *Grafo*

## Item **1** : Criação do grafo a partir da lista de adjancências
A implementação do grafo que utiliza a lista de adjacência como estrutura de dados subjacente foi feita através das classes *ListaAdjacencia* e *VerticeAdjacencia*.
Aqui, contamos com o método *ler_grafo*, responsável por realizar a leitura do grafo descrito no filepath fornecido como
entrada e fazer a construção do grafo. Essa construção consiste em, para cada par de vértices lidos, instanciar os objetos
vértices correspondentes, e inserí-los no grafo via o método *adicionar_vertice*, caso já não pertençam ao grafo. Se já pertencerem, é feita a
criação de novas arestas via a função *adicionar_vizinho* encapsulada na classe *VerticeAdjacencia*.
Maior detalhamento sobre o método de adição será fornecido na resolução do item **9**
Esta implementação da classe já conta, também, com a implementação do método de remoção de vértice ao grafo.



In [1]:
from collections import Counter
class VerticeAdjacencia:
    def __init__(self, label, vizinhos = None):
        self.label = label
        # self.predecessor = predecessor if predecessor else None
        vizinhos = vizinhos if vizinhos else []
        # TODO : Remover
        self.vizinhos = set(vizinhos)
    def adicionar_vizinho(self, vizinho):
        self.vizinhos.add(vizinho)
        vizinho.vizinhos.add(self)
    def __str__(self):
        return f"Label : {self.label}"

# Nesta implementacao o grafo consistirá em um dict cujos valores sao lists (implementacoes de linkedlista=)
class ListaAdjacencia:
    def __init__(self, nome,  caminho_arquivo : str = None):
        self.lista = dict()
        self.labels = dict()
        self.nome = nome
        self.tamanho = 0
        if caminho_arquivo:
            vertices = self.ler_grafo(caminho_arquivo=caminho_arquivo)
            for vertice in vertices:
                self.adicionar_vertice(vertice)

    def adicionar_vertice(self, novo_vertice): # O(m), tq m eh numero de vertices vizinhos ao novo_vertice JA PRESENTES NO GRAFO
        self.lista[novo_vertice] = set()
        self.labels[novo_vertice.label] = novo_vertice
        # Ver os vizinhos do vertice que vai ser adicionado
        # adicionar o novo vertice a lista de colisao de cada um de seus vizinhos

        for vizinho in novo_vertice.vizinhos:
            # Fazemos a checagem se o a aresta ja foi contabilizada para incrementar o atributo 'tamanho'
            # Para isso, checamos se apos a insercao do vizinho
            # tamanho_set_antes =  len(self.lista[novo_vertice])
            vizinho.adicionar_vizinho(novo_vertice)
            self.lista[novo_vertice].add(vizinho)
            if not self.lista.get(vizinho):
                self.lista[vizinho] = set()
            tamanho_antes = len(self.lista[vizinho])
            self.lista[vizinho].add(novo_vertice)
            if tamanho_antes < len(self.lista[vizinho]):
                self.tamanho += 1

    def ler_grafo(self, caminho_arquivo : str):
        with open(caminho_arquivo, "r") as grafo_arquivo:
            linhas = grafo_arquivo.readlines()
            ordem = int(linhas[0])
            vertices = dict()
            def extrair_labels(linha: str):
                label1, label2 = linha.strip().split(",")
                if label1.isnumeric():
                    label1 = int(label1)
                    label2 = int(label2)
                return label1, label2

            for linha in linhas[1:]:
                label_1, label_2 = extrair_labels(linha)
                # Checa se os vertices recebidos na linha ja existem
                vert_1, vert_2 = vertices.get(label_1, None), vertices.get(label_2, None)
                if vert_1 and vert_2:
                    vert_1.adicionar_vizinho(vert_2)
                elif vert_1:
                    vert_2 = VerticeAdjacencia(label=label_2, vizinhos=[vert_1])
                    vertices[label_2] = vert_2
                elif vert_2:
                    vert_1 = VerticeAdjacencia(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                else:
                    vert_2 = VerticeAdjacencia(label=label_2)
                    vert_1 = VerticeAdjacencia(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                    vertices[label_2] = vert_2
            return vertices.values()
    def remover_vertice(self, vertice_a_ser_removido):
        if vertice_a_ser_removido not in self.lista:
            return "Vertice nao pertence ao grafo"
        self.lista.pop(vertice_a_ser_removido)  # O(1)
        for vizinho in vertice_a_ser_removido.vizinhos: # O(m)
            del self.lista[vizinho][vertice_a_ser_removido] # O(1)
            self.tamanho =- 1

    def get_vertices(self):
        return self.lista.keys()

    def get_vertice(self,label):
        return self.labels[label]


    def get_adjacentes(self, label_vertice):
        vertice = self.labels[label_vertice]
        return set(vertice.vizinhos)

    def __str__(self):
        if len(self.lista) ==0:
            return "<ListaAdjacencia Vazio>"
        strings = [f"{vertice.label} -> {list(map(lambda vizinho : vizinho.label, self.lista[vertice]))}" for vertice in self.lista.keys()]
        return '\n'.join(strings)


grafo = ListaAdjacencia("Grafo 1",caminho_arquivo="./data/GRAFO_1.txt")
print(grafo)


a -> ['b']
b -> ['d', 'e', 'f', 'a', 'c']
c -> ['b', 'd', 'e', 'i']
d -> ['b', 'h', 'c']
e -> ['b', 'c']
f -> ['b']
i -> ['c']
h -> ['d']


## Item **2** : Implementação do Grafo usando *Matriz de adjacência*
A implementação usando matriz de adjacência tem a mesma estrutura do item anterior, é feita a leitura via método *ler_grafo* e adição dos vértices.
Nesta implementação, mantemos o a classe Vertice, mas aqui ela é usada apenas para encapsular o dado vértice e seus vizinhos, para que sirva de entrada ao método *adicionar_vertice*. A estrutura do grafo é mantida integralmente na matriz

In [2]:
from collections import Counter
from functools import reduce

# Implementacao da classe vertice para de grafos utilizando matriz de adjacencia
class VerticeMatriz:
    def __init__(self, label : str = "", vizinhos = None):
        self.label = label
        self.indice = None
        # O Counter associa um objeto a um contador numérico via hash table, aqui será usado para contar o número de
        # arestas entre este vértice e seu respectivo vizinho, para o caso de recebermos multigrafos
        self.vizinhos = Counter()
        if vizinhos:
            for vizinho in vizinhos:
                self.adicionar_vizinho(vizinho)

    def adicionar_vizinho(self, vertice_vizinho):
        self.vizinhos[vertice_vizinho] += 1
        vertice_vizinho.vizinhos[self] += 1

class MatrizAdjacencia:
    def __init__(self, nome : str ,caminho_arquivo : str = None, vertices = None): # O(n^2)
        self.nome = nome # Nome do grafo, pra proposito de impressao
        self.indices = dict() # hash table associando os vertices a cada indice da matriz, para agilizar consultas
        self.matriz = []
        self.labels = dict() # Hash table para armazenar os vértices associadas a cada label
        self.tamanho = 0
        if vertices:
            for vertice in vertices:
                self.adicionar_vertice(vertice)
        else:
            if caminho_arquivo:
                for vertice in self.ler_grafo(caminho_arquivo):
                    self.adicionar_vertice(vertice)

    def ler_grafo(self, caminho_arquivo : str):
        with open(caminho_arquivo, "r") as grafo_arquivo:
            linhas = grafo_arquivo.readlines()
            ordem = int(linhas[0])
            vertices = dict()
            def extrair_labels(linha: str):
                label1, label2 = linha.strip().split(",")
                if label1.isnumeric():
                    label1 = int(label1)
                    label2 = int(label2)
                return label1, label2

            for linha in linhas[1:]:
                label_1, label_2 = extrair_labels(linha)

                # Checa se os vertices recebidos na linha ja existem
                vert_1, vert_2 = vertices.get(label_1, None), vertices.get(label_2, None)
                if vert_1 and vert_2:
                    vert_1.adicionar_vizinho(vert_2)
                elif vert_1:
                    vert_2 = VerticeMatriz(label=label_2)
                    vert_1.adicionar_vizinho(vert_2)
                    vertices[label_2] = vert_2
                elif vert_2:
                    vert_1 = VerticeMatriz(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                else:
                    vert_2 = VerticeMatriz(label=label_2)
                    vert_1 = VerticeMatriz(label=label_1, vizinhos=[vert_2])
                    vertices[label_1] = vert_1
                    vertices[label_2] = vert_2
            return sorted(vertices.values(), key=lambda key : key.label)


    def adicionar_vertice(self, vertice : VerticeMatriz): # O(n)
        # Adiciona o novo label
        self.labels[vertice.label] = vertice
        # Caso a matriz esteja vazia
        if len(self.matriz) == 0:
            vertice.indice = 0
            # Adicionamos à linha o valor referente ao número de laços presentes neste vértice
            self.matriz.append(list([vertice.vizinhos[vertice]]))
            self.indices[0] = vertice
            self.tamanho += vertice.vizinhos[vertice]
            return

        # Linha que será adcioanada à matriz
        nova_linha = []
        # Adiciona o índice do novo vértice à hash table
        self.indices[len(self.matriz)] = vertice # O(1)
        vertice.indice = len(self.matriz) # O(1)
        # Itera por cada linha da matriz adicionando uma nova coluna com valor igual ao número de arestas entre
        # o novo vértice e o da linha atual
        for indice, linha in enumerate(self.matriz): # O(n)
            vertice_atual = self.indices[indice] # O(1)
            quantidade = vertice.vizinhos.get(vertice_atual, 0) # O(1)
            linha.append(quantidade) # O(1)
            nova_linha.append(quantidade) #O(1)
            self.tamanho += quantidade

        # Atualiza a nova linha para o caso do vértice adicionado apresentar laços
        quantidade_arestas_proprias = vertice.vizinhos.get(vertice, 0) # O(1)
        self.tamanho += quantidade_arestas_proprias
        nova_linha.append(quantidade_arestas_proprias) # O(1)
        self.matriz.append(nova_linha) #O(1)

    def get_vertices(self):
        return self.indices.values()

    def get_adjacentes(self, label_vertice):
        vertice = self.labels[label_vertice]
        indice_na_matriz = vertice.indice
        linha = self.matriz[indice_na_matriz]
        # É feita a filtragem dos elementos
        vizinhos_map = map(lambda i : self.indices[i], filter(lambda indice : linha[indice] > 0, range(0, len(linha))))
        return set(vizinhos_map)

    def get_vertice(self, label_vertice):
        return self.labels[label_vertice]

    def remover_vertice(self, label_vertice): # O(n + n*n) = O(n^2)
        if label_vertice not in self.labels:
            raise ValueError("Vértice não faz parte do grafo") # O(1)

        vertice = self.labels[label_vertice]
        indice_a_remover = vertice.indice
        print(self.indices.keys())
        for i, linha in enumerate(self.matriz): # O(n)
            if i == indice_a_remover:
                continue
            if linha[indice_a_remover] != 0:    # O(1)
                # Remove o vertice removido da lista de vizinhos dos outros vertices
                self.indices[i].vizinhos.pop(vertice) # O(1)
                self.tamanho -= 1
            # Atualiza os valores dos indices associados aos vertices para refletir as novas posicoes
            if i > indice_a_remover:
                self.indices[i - 1] = self.indices[i]
                self.indices[i - 1].indice = i - 1
                self.indices.pop(i)
            linha.pop(indice_a_remover) # O(n)

        # Remove da matriz a linha referente ao vértice removido
        self.matriz.pop(indice_a_remover)

    def __str__(self):
        return str(self.matriz)

In [3]:
grafo = MatrizAdjacencia("Grafo - 1", "./data/GRAFO_2.txt")
print(grafo)

[[0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]


## Item **3** - Implementação do Grafo usando da Matriz de Incidência

In [4]:
class MatrizIncidencia:
    def __init__(self, nome, caminho_arquivo=None):
        self.nome = nome # para melhor identificação
        self.vertices = {}  # hash table para mapear label do vértice para o índice
        self.arestas = []   # hash table representando as arestas
        self.matriz = []

        if caminho_arquivo:
            self.ler_grafo(caminho_arquivo)

    def ler_grafo(self, caminho_arquivo):
        with open(caminho_arquivo, "r") as f:
            linhas = f.readlines()[1:] # para ignorar a primeira linha com o número de vértices

            # Primeiro, identifica todos os vértices únicos para definir o tamanho da matriz
            vertices_labels = set()
            for linha in linhas:
                v1_label, v2_label = linha.strip().split(',')
                vertices_labels.add(v1_label)
                vertices_labels.add(v2_label)

            for label in sorted(list(vertices_labels)): # ordena para consistência
                self.adicionar_vertice(label)

            # adiciona as arestas e constrói a matriz
            for linha in linhas:
                v1_label, v2_label = linha.strip().split(',')
                self.adicionar_aresta(v1_label, v2_label)

    def adicionar_vertice(self, label):
        if label not in self.vertices:
            self.vertices[label] = len(self.vertices)
            # adiciona uma nova linha (vazia por enquanto) para o novo vértice
            self.matriz.append([])

    def adicionar_aresta(self, v1_label, v2_label):
        # adiciona a nova aresta
        self.arestas.append((v1_label, v2_label))
        indice_aresta = len(self.arestas) - 1

        # adiciona a nova coluna à matriz (representada por adicionar um elemento a cada linha)
        for i in range(len(self.vertices)):
            self.matriz[i].append(0)

        # preenche os valores da nova coluna
        i1 = self.vertices[v1_label]
        i2 = self.vertices[v2_label]

        if i1 == i2:  # laço
            self.matriz[i1][indice_aresta] = 2
        else:  # grafo não direcionado
            self.matriz[i1][indice_aresta] = 1
            self.matriz[i2][indice_aresta] = 1

    def __str__(self):
       return str(self.matriz)

grafo_incidencia = MatrizIncidencia("GRAFO 1", "./data/GRAFO_1.txt")
print(grafo_incidencia)

[[1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0, 0, 0, 0], [0, 1, 0, 0, 0, 1, 1, 1, 0], [0, 0, 1, 0, 0, 1, 0, 0, 1], [0, 0, 0, 1, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0]]


## Item **4** : Conversão de matriz de adjacência para lista de Adjacências e vice-versa.
A implementação é bastante simples: obtemos os vértices do grafo inicial a partir do método *get_vertices* e
fazemos a inserção destes no novo grafo vazio - que utiliza a estrutura de dados procurada.

In [5]:
def converter_lista_pra_matriz(grafo_inicial : ListaAdjacencia) -> MatrizAdjacencia:

    vertices_convertidos = dict()
    for vertice in grafo_inicial.get_vertices():
        vertice_convertido = VerticeMatriz(label=vertice.label)
        for vizinho in vertice.vizinhos:
            vizinho_convertido = vertices_convertidos.get(vizinho.label, None)
            if not vizinho_convertido:
                vizinho_convertido = VerticeMatriz(label=vizinho.label)
                vertices_convertidos[vizinho.label] = vizinho_convertido
            vertice_convertido.adicionar_vizinho(vizinho_convertido)
            # vizinho_convertido.adicionar_vizinho(vertice_convertido)
        vertices_convertidos[vertice_convertido.label] = vertice_convertido
    novo = MatrizAdjacencia(nome=grafo_inicial.nome, vertices=sorted(list(vertices_convertidos.values()), key=lambda key : key.label))
    return  novo

def converter_matriz_pra_lista(grafo_inicial : MatrizAdjacencia) -> ListaAdjacencia:
    novo_grafo = ListaAdjacencia(grafo_inicial.nome)
    for vertice in grafo_inicial.get_vertices():
        novo_grafo.adicionar_vertice(vertice)
    return novo_grafo

matriz = MatrizAdjacencia(nome="Grafo que sera convertido", caminho_arquivo="./data/GRAFO_2.txt")
print(matriz)
lista = converter_matriz_pra_lista(matriz)
print(converter_matriz_pra_lista(matriz))
matriz = converter_lista_pra_matriz(lista)
print(converter_lista_pra_matriz(lista))


[[0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]
1 -> [3, 2, 6]
2 -> [3, 5, 1, 6]
3 -> [2, 1, 4]
6 -> [2, 1, 7]
5 -> [2, 4]
4 -> [3, 5]
7 -> [6]
8 -> [10, 11, 9]
9 -> [8, 10]
10 -> [8, 9]
11 -> [8]
[[0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0], [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0], [1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0], [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]


## Item **5** : Função que calcula o grau de cada vértice.

In [6]:
from functools import reduce
def calcular_grau_vertice_matriz(grafo, label_vertice) -> int:
    # Obtem o objeto do vertice de entrada, para que possamos extrair seu indice na matriz
    vertice = grafo.get_vertice(label_vertice)
    # Faz uma operacao reducao somando todos os valores da linha correspondente ao vertice de entrada
    # Isso nos fornece o numero de arestas no qual este vertice eh terminal, e.g seu grau
    return reduce(lambda a,b : a+b, grafo.matriz[vertice.indice])
grafo = MatrizAdjacencia("Grafo", "./data/GRAFO_2.txt")
print(calcular_grau_vertice_matriz(grafo, 3))


3


## Item **6** : Função que determina se dois vértices são adjacentes.

In [7]:
# Implementacao para a Matriz e Lista de adjacencia
def sao_adjacentes(grafo, label_vert_1, label_vert_2) -> bool : # O(1)
    vert_1, vert_2 = grafo.get_vertice(label_vert_1) , grafo.get_vertice(label_vert_2) # O(1)
    return vert_2 in vert_1.vizinhos or vert_1 in vert_2.vizinhos


grafo = MatrizAdjacencia("Grafo", "./data/GRAFO_2.txt")
print(sao_adjacentes(grafo, 1, 6))


True


## Itens **7** e **8** : Função que determina os números totais de vértices e arestas
Para obter a ordem dos grafos implementados com lista de adjacência, basta obter o tamanho do seu atributo *lista*, já para a matriz, o tamanho de *matriz*.
A obtenção do tamanho para os grafos nessas estruturas é um pouco mais complexa. A solução ingênua seria: para a lista, iterar por cada vértice e incrementar o tamanho para cada vizinho encontrado, enquanto o para a matriz basta checar as arestas em cada linha.
Para dirimir a complexidade de invocar este método, o atributo tamanho é ajustado a cada chamada dos métodos *adicionar_vertice* e *remover_vertice*.
Para a lista, ao adicionar vértice, é checado se as novas arestas conectadas a ele já foram contabilizadas - no caso de seu vértice adjacente conectado a esta aresta já ter sido adicionado - se não, incrementao tamanho

In [8]:
def get_tamanho(grafo):
    return grafo.tamanho


def get_ordem_lista(grafo):
    return len(grafo.lista)

def get_ordem_matriz_adjacencia(grafo):
    return len(grafo.matriz)

MatrizAdjacencia.get_ordem = get_ordem_matriz_adjacencia
ListaAdjacencia.get_ordem = get_ordem_lista

MatrizAdjacencia.get_tamanho = get_tamanho
ListaAdjacencia.get_tamanho = get_tamanho
grafo = ListaAdjacencia("Grafo 1 - Lista de Adjacencia", "./data/GRAFO_2.txt")
print(grafo)
print(f"Ordem do grafo ({grafo.nome}): {grafo.get_ordem()}")
print(f"Tamanho do grafo ({grafo.nome}): {grafo.get_tamanho()}")

grafo_mat = MatrizAdjacencia("Grafo 1 - Matriz de Adjacencia", "./data/GRAFO_2.txt")
print(f"Ordem do grafo ({grafo_mat.nome}): {grafo_mat.get_ordem()}")
print(f"Tamanho do grafo ({grafo_mat.nome}): {grafo_mat.get_tamanho()}")


1 -> [2, 3, 6]
6 -> [1, 7, 2]
2 -> [1, 6, 3, 5]
3 -> [4, 1, 2]
5 -> [4, 2]
4 -> [5, 3]
7 -> [6]
8 -> [10, 11, 9]
9 -> [10, 8]
10 -> [8, 9]
11 -> [8]
Ordem do grafo (Grafo 1 - Lista de Adjacencia): 11
Tamanho do grafo (Grafo 1 - Lista de Adjacencia): 13
Ordem do grafo (Grafo 1 - Matriz de Adjacencia): 11
Tamanho do grafo (Grafo 1 - Matriz de Adjacencia): 13


## Item **11** : Função que determina se um grafo é conexo ou não.

In [9]:
def eh_conexo_lista(grafo):
    "Verifica se um grafo é conexo usando lista como fila."
    todos_os_vertices = list(grafo.get_vertices())
    if not todos_os_vertices:
        return False  # grafo vazio n é conexo

    fila = [todos_os_vertices[0]]  # lista
    visitados = {todos_os_vertices[0]} # armazena os vertices visitados

    while fila:
        vertice_atual = fila.pop(0)  # remove do inicio da lista
        for vizinho in grafo.get_adjacentes(vertice_atual.label): # se o vizinho ainda n foi visitado, adiciona a fila /conjunto

            if vizinho not in visitados:
                visitados.add(vizinho)
                fila.append(vizinho)

    return len(visitados) == len(todos_os_vertices)  # caso todos os vertices forem visitados, o grafo é conexo

grafo_conexo = ListaAdjacencia("Grafo 1", "./data/GRAFO_1.txt")
grafo_desconexo = ListaAdjacencia("Grafo 2", "./data/GRAFO_2.txt")

print(f"O Grafo 1 é conexo? {eh_conexo_lista(grafo_conexo)}")
print(f"O Grafo 2 é conexo? {eh_conexo_lista(grafo_desconexo)}")

O Grafo 1 é conexo? True
O Grafo 2 é conexo? False


## Item **14** - Busca em Profundidade, com determinação de arestas de retorno, a partir de um vértice em específico

In [10]:
# TODO : Retornar árvore de profundidade e checar se precisa fazer o percurso na ordem
def depth_first_search(grafo, label_vertice_inicial, ordenado : bool = None):
    pilha = list()
    vertice_inicial = grafo.get_vertice(label_vertice_inicial)
    # AS aresta serao representadas por 2-uplas
    arestas_visitadas = set()
    arestas_de_retorno = set()
    # Set para monitorar quais vertices ainda nao foram visitados
    vertices_nao_visitados = set(grafo.get_vertices())
    # Set para monitorar quais vertices ainda nao foram explorados
    vertices_nao_explorados = set(grafo.get_vertices())
    print(f"## Percurso em profundidade em '{grafo.nome}'")
    print(f"Vertice visitado: {vertice_inicial.label}, predecessor: X", )
    pilha.append(vertice_inicial)
    vertices_nao_visitados.remove(vertice_inicial)
    vizinhos_vertices = dict()
    while vertices_nao_explorados:
        if len(pilha) == 0:
            menor_label = min([i.label for i in vertices_nao_visitados])
            nova_raiz = grafo.get_vertice(min([i.label for i in vertices_nao_visitados]))
            pilha.append(nova_raiz)
            print(f"Vertice visitado: {nova_raiz.label}, predecessor: X", )
            vertices_nao_visitados.remove(nova_raiz)
        u = pilha[-1]
        if u not in vizinhos_vertices:
            if ordenado:
                # TODO : Refatorar
                labels_ordenadas = sorted([i.label for i in grafo.get_adjacentes(u.label)], reverse=True)
                vizinhos_vertices[u] = [grafo.get_vertice(v) for v in labels_ordenadas]
            else:
                vizinhos_vertices[u] = grafo.get_adjacentes(u.label)

        v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None
        if not v:
            pilha.pop()
            vertices_nao_explorados.remove(u) # O vertice eh marcado como explorado
        elif v in vertices_nao_visitados:
            vertices_nao_visitados.remove(v)
            print(f"Vertice visitado: {v.label}, predecessor : {u.label}", )
            pilha.append(v)
            arestas_visitadas.add((u,v))
        else:
            if v in vertices_nao_explorados and pilha[-2] != v: # Checa se é adjaccente na pilha
                if (v,u) not in arestas_visitadas:
                    arestas_visitadas.add((u,v))
                    arestas_de_retorno.add((u,v))
                    print("Nova aresta de retorno: ", (u.label,v.label))

    # Printa arestas de retorno
    arestas_de_retorno = map(lambda aresta : (aresta[0].label, aresta[1].label), arestas_de_retorno)
    print(f"Arestas de retorno: {list(arestas_de_retorno)}")

In [11]:
from IPython.display import display_markdown
display_markdown("**Busca em profundidade em grafo conexo**")
grafo1 = MatrizAdjacencia("Grafo Matriz", "./data/GRAFO_1.txt")
depth_first_search(grafo=grafo1, label_vertice_inicial="a", ordenado=True)

grafo1 = ListaAdjacencia("Grafo Lista", "./data/GRAFO_1.txt")
depth_first_search(grafo=grafo1, label_vertice_inicial="a", ordenado=True)



## Percurso em profundidade em 'Grafo Matriz'
Vertice visitado: a, predecessor: X
Vertice visitado: b, predecessor : a
Vertice visitado: c, predecessor : b
Vertice visitado: d, predecessor : c
Nova aresta de retorno:  ('d', 'b')
Vertice visitado: h, predecessor : d
Vertice visitado: e, predecessor : c
Nova aresta de retorno:  ('e', 'b')
Vertice visitado: i, predecessor : c
Vertice visitado: f, predecessor : b
Arestas de retorno: [('d', 'b'), ('e', 'b')]
## Percurso em profundidade em 'Grafo Lista'
Vertice visitado: a, predecessor: X
Vertice visitado: b, predecessor : a
Vertice visitado: c, predecessor : b
Vertice visitado: d, predecessor : c
Nova aresta de retorno:  ('d', 'b')
Vertice visitado: h, predecessor : d
Vertice visitado: e, predecessor : c
Nova aresta de retorno:  ('e', 'b')
Vertice visitado: i, predecessor : c
Vertice visitado: f, predecessor : b
Arestas de retorno: [('d', 'b'), ('e', 'b')]


In [12]:

display_markdown("**Busca em profundidade em grafo nao conexo**")
grafo1 = ListaAdjacencia("Grafo 2 - Lista de Adjacencia", "./data/GRAFO_2.txt")
depth_first_search(grafo=grafo1, label_vertice_inicial=1, ordenado=True)

grafo1 = MatrizAdjacencia("Grafo 2 - Matriz de Adjacencia", "./data/GRAFO_2.txt")
depth_first_search(grafo=grafo1, label_vertice_inicial=1, ordenado=True)

## Percurso em profundidade em 'Grafo 2 - Lista de Adjacencia'
Vertice visitado: 1, predecessor: X
Vertice visitado: 2, predecessor : 1
Vertice visitado: 3, predecessor : 2
Nova aresta de retorno:  (3, 1)
Vertice visitado: 4, predecessor : 3
Vertice visitado: 5, predecessor : 4
Nova aresta de retorno:  (5, 2)
Vertice visitado: 6, predecessor : 2
Nova aresta de retorno:  (6, 1)
Vertice visitado: 7, predecessor : 6
Vertice visitado: 8, predecessor: X
Vertice visitado: 9, predecessor : 8
Vertice visitado: 10, predecessor : 9
Nova aresta de retorno:  (10, 8)
Vertice visitado: 11, predecessor : 8
Arestas de retorno: [(10, 8), (3, 1), (6, 1), (5, 2)]
## Percurso em profundidade em 'Grafo 2 - Matriz de Adjacencia'
Vertice visitado: 1, predecessor: X
Vertice visitado: 2, predecessor : 1
Vertice visitado: 3, predecessor : 2
Nova aresta de retorno:  (3, 1)
Vertice visitado: 4, predecessor : 3
Vertice visitado: 5, predecessor : 4
Nova aresta de retorno:  (5, 2)
Vertice visitado: 6, predecessor : 

# Item 15 - Determinar articulações e blocos de biconectividade usando lowpt

In [13]:
def dps_lowpt(grafo, label_vertice_inicial, ordenado: bool = None):
    pilha = list()
    vertice_inicial = grafo.get_vertice(label_vertice_inicial)
    # AS aresta serao representadas por 2-uplas
    arestas_visitadas = set()
    arestas_de_retorno = set()
    # Set para monitorar quais vertices ainda nao foram visitados
    vertices_nao_visitados = set(grafo.get_vertices())
    # Set para monitorar quais vertices ainda nao foram explorados
    vertices_nao_explorados = set(grafo.get_vertices())
    # Dicionário de pais (vértice -> pai)
    pais = {}
    # Dicionário de lowpts (vértice -> lowpt)
    lowpts = {}
    # Set de demarcadores
    demarcadores = set()
    # Dicionario de raizes de raizes e qnt de filhos (raiz -> qnt_filhos)
    raizes = {}
    # Set de articulacoes
    articulacoes = set()
    # Determinação dos blocos
    pilha_blocos = []
    blocos_biconexos = []
    print(f"## Percurso em profundidade em '{grafo.nome}'") 
    print(f"Vertice visitado: {vertice_inicial.label}, predecessor: X", )
    pilha.append(vertice_inicial)
    vertices_nao_visitados.remove(vertice_inicial)
    pais[vertice_inicial] = None
    lowpts[vertice_inicial] = vertice_inicial
    raizes[vertice_inicial] = 0
    vizinhos_vertices = dict()
    while vertices_nao_explorados:
        if len(pilha) == 0:
            # Novo componente conexo
            menor_label = min([i.label for i in vertices_nao_visitados])
            nova_raiz = grafo.get_vertice(menor_label)
            pilha.append(nova_raiz)
            print(f"Vertice visitado: {nova_raiz.label}, predecessor: X")
            vertices_nao_visitados.remove(nova_raiz)
            pais[nova_raiz] = None
            lowpts[nova_raiz] = nova_raiz
            raizes[nova_raiz] = 0
        
        u = pilha[-1]
        
        if u not in vizinhos_vertices:
            if ordenado:
                labels_ordenadas = sorted([i.label for i in grafo.get_adjacentes(u.label)], reverse=True)
                vizinhos_vertices[u] = [grafo.get_vertice(v) for v in labels_ordenadas]
            else:
                vizinhos_vertices[u] = grafo.get_adjacentes(u.label)
        
        v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None
        
        if not v:
            # Vértice u será removido da pilha - CALCULAR LOWPT AQUI
            pilha.pop()
            vertices_nao_explorados.remove(u)
            
            # Inicialmente, lowpt é o próprio vértice u
            current_lowpt = u
            
            # Verifica se há arestas de retorno que saem do vértice
            for aresta in arestas_de_retorno:
                if aresta[0] == u:
                    current_lowpt = aresta[1]  # vértice alcançado pela aresta de retorno é o novo lowpt
            
            # Verifica lowpts dos filhos de u
            for w, pai_w in pais.items():
                if pai_w == u: # Se achar um filho de u
                    pos_new = None # posição na pilha do novo possivel lowpt
                    pos_current = len(pilha) + 1 # posição na pilha do atual lowpt (inicializa com o valor maximo possivel para caso o lowpt seja ele mesmo, caso não seja, será tratado durante a iteração)
                    for i, vertice in enumerate(pilha): # percorre a pilha 
                        if vertice == lowpts[w] and vertice != w:
                            # caso o vertice na pilha seja o lowpt de w (filho de u) e seja diferente do proprio w, pos_new sera o indice (local na pilha)
                            pos_new = i
                        if vertice == current_lowpt:
                            # caso ache o atual lowpt na pilha (ou seja, o lowpt atual é diferente do proprio vertice) adiciona a posição correta
                            pos_current = i
                    if pos_new is not None: # Checa se achou um possivel candidato para novo lowpt
                        if pos_new < pos_current: # Se achou, checa se o novo candidato está mais em baixo na pilha que o atual
                            current_lowpt = lowpts[w] # Caso esteja, o novo lowpt sera o lowpt do filho
            
            lowpts[u] = current_lowpt

            if lowpts[u] == u or lowpts[u] == pais[u]: # Checa se u é demarcador
                demarcadores.add(u)
            
            if u in demarcadores and pais[u] is not None: # Se u for demarcador e não for raiz (tem pai)
                if pais[u] not in raizes: # Checa se o pai é raiz
                    articulacoes.add(pais[u]) # Caso não seja, adiciona o pai como articulação
                bloco = set() 
                while pilha_blocos: # Percore a pilha de arestas adicionando os vertices da subarvore
                    x, y = pilha_blocos.pop()
                    bloco.add(x)
                    bloco.add(y)
                    if (x, y) == (u, pais[u]) or (y, x) == (u, pais[u]): # Se encontra a aresta de u com seu pai, termina 
                        break
                blocos_biconexos.append(bloco) 
            elif u in raizes: # Checa se u é raiz e tem mais de um filho
                if raizes[u] >= 2:
                    articulacoes.add(u) # Caso sim, adiciona como articulacao
            
            print(f"Vértice {u.label} explorado -> lowpt: {current_lowpt.label}")
            
        elif v in vertices_nao_visitados:
            vertices_nao_visitados.remove(v)
            print(f"Vertice visitado: {v.label}, predecessor: {u.label}")
            pilha.append(v)
            arestas_visitadas.add((u, v))
            pilha_blocos.append((u, v))
            pais[v] = u
            if u in raizes:
                raizes[u] += 1
            lowpts[v] = v 
            
        else:
            if v in vertices_nao_explorados and (len(pilha) < 2 or pilha[-2] != v):
                if (v, u) not in arestas_visitadas:
                    arestas_visitadas.add((u, v))
                    arestas_de_retorno.add((u, v))
                    pilha_blocos.append((u, v))
                    print("Nova aresta de retorno: ", (u.label, v.label))

    # Printa arestas de retorno
    arestas_de_retorno_labels = [(aresta[0].label, aresta[1].label) for aresta in arestas_de_retorno]
    print(f"Arestas de retorno: {list(arestas_de_retorno_labels)}")
    
    # Resultados finais
    print("\n=== RESULTADOS FINAIS ===")
    demarcadores_labels = [vertice.label for vertice in demarcadores]
    print(f"\nDemarcadores: {list(demarcadores_labels)}")
    
    articulacoes_labels = [vertice.label for vertice in articulacoes]
    print(f"Articulações: {list(articulacoes_labels)}")

    print("\n=== BLOCOS BICONEXOS ===")
    for i, bloco in enumerate(blocos_biconexos, start=1):
        print(f"Bloco {i}: {[v.label for v in bloco]}")

    for vertice in grafo.get_vertices():
        pai_label = pais[vertice].label if vertice in pais and pais[vertice] else "X"
        print(f"Vértice {vertice.label} (pai: {pai_label}) -> lowpt: {lowpts[vertice].label}")

In [14]:
display_markdown("**Busca em profundidade em grafo nao conexo com determinação de blocos e articulações**")
grafo1 = ListaAdjacencia("Grafo 2 - Lista de Adjacencia", "./data/GRAFO_2.txt")
dps_lowpt(grafo=grafo1, label_vertice_inicial=1, ordenado=True)

## Percurso em profundidade em 'Grafo 2 - Lista de Adjacencia'
Vertice visitado: 1, predecessor: X
Vertice visitado: 2, predecessor: 1
Vertice visitado: 3, predecessor: 2
Nova aresta de retorno:  (3, 1)
Vertice visitado: 4, predecessor: 3
Vertice visitado: 5, predecessor: 4
Nova aresta de retorno:  (5, 2)
Vértice 5 explorado -> lowpt: 2
Vértice 4 explorado -> lowpt: 2
Vértice 3 explorado -> lowpt: 1
Vertice visitado: 6, predecessor: 2
Nova aresta de retorno:  (6, 1)
Vertice visitado: 7, predecessor: 6
Vértice 7 explorado -> lowpt: 7
Vértice 6 explorado -> lowpt: 1
Vértice 2 explorado -> lowpt: 1
Vértice 1 explorado -> lowpt: 1
Vertice visitado: 8, predecessor: X
Vertice visitado: 9, predecessor: 8
Vertice visitado: 10, predecessor: 9
Nova aresta de retorno:  (10, 8)
Vértice 10 explorado -> lowpt: 8
Vértice 9 explorado -> lowpt: 8
Vertice visitado: 11, predecessor: 8
Vértice 11 explorado -> lowpt: 11
Vértice 8 explorado -> lowpt: 8
Arestas de retorno: [(6, 1), (10, 8), (5, 2), (3, 1)]



In [15]:
display_markdown("**Busca em profundidade em grafo conexo com determinação de blocos e articulações**")
grafo1 = MatrizAdjacencia("Grafo 1 - Matriz de Adjacencia", "./data/GRAFO_1.txt")
dps_lowpt(grafo=grafo1, label_vertice_inicial='a', ordenado=True)

## Percurso em profundidade em 'Grafo 1 - Matriz de Adjacencia'
Vertice visitado: a, predecessor: X
Vertice visitado: b, predecessor: a
Vertice visitado: c, predecessor: b
Vertice visitado: d, predecessor: c
Nova aresta de retorno:  ('d', 'b')
Vertice visitado: h, predecessor: d
Vértice h explorado -> lowpt: h
Vértice d explorado -> lowpt: b
Vertice visitado: e, predecessor: c
Nova aresta de retorno:  ('e', 'b')
Vértice e explorado -> lowpt: b
Vertice visitado: i, predecessor: c
Vértice i explorado -> lowpt: i
Vértice c explorado -> lowpt: b
Vertice visitado: f, predecessor: b
Vértice f explorado -> lowpt: f
Vértice b explorado -> lowpt: b
Vértice a explorado -> lowpt: a
Arestas de retorno: [('e', 'b'), ('d', 'b')]

=== RESULTADOS FINAIS ===

Demarcadores: ['f', 'a', 'b', 'c', 'h', 'i']
Articulações: ['d', 'c', 'b']

=== BLOCOS BICONEXOS ===
Bloco 1: ['d', 'h']
Bloco 2: ['i', 'c']
Bloco 3: ['b', 'c', 'e', 'd']
Bloco 4: ['b', 'f']
Bloco 5: ['a', 'b']
Vértice a (pai: X) -> lowpt: a
Vérti

## Item **13** - Busca em Largura, a partir de um vértice específico.

In [ ]:
from collections import deque
def width_first_search(grafo, label_vertice_inicial, ordenado : bool = None):
    fila = deque()
    vertices_nao_visitados = set(grafo.get_vertices()) # Pra o caso do grafo nao ser conexo
    arestas_visitadas = set()
    vertice_inicial = grafo.get_vertice(label_vertice_inicial)
    vertices_nao_visitados.remove(vertice_inicial)
    print("## Percurso em Largura")
    print("Vertice inicial: ", vertice_inicial.label)
    fila.append(vertice_inicial)
    vizinhos_vertices = dict()
    eh_conexo = False
    while len(vertices_nao_visitados) > 0:
        if len(fila) == 0:
            nova_raiz =vertices_nao_visitados.pop()
            print(f"Vertice visitado: {nova_raiz.label}; Predecessor : X",)
            fila.append(nova_raiz)
            eh_conexo = True
        u = fila.popleft()
        # Se os vertices adjacentes de u nao estiverem em nosso dict que os armazena, os insere
        if ordenado:
            labels_ordenadas = sorted([i.label for i in grafo.get_adjacentes(u.label)], reverse=True)
            vizinhos_vertices[u] = [grafo.get_vertice(v) for v in labels_ordenadas]
        else:
            vizinhos_vertices[u] = grafo.get_adjacentes(u.label)
        v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None
        while v: # O(n) -> n : numero de vizinhos do vertice u
            if v in vertices_nao_visitados:
                vertices_nao_visitados.remove(v)
                fila.append(v)
                print(f"Vertice visitado: {v.label}; Predecessor : {u.label}",)
            v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None

In [ ]:
grafo_matriz = ListaAdjacencia("Grafo Matriz", "./data/GRAFO_2.txt")
print(grafo_matriz)
width_first_search(grafo_matriz, "1")

1 -> ['3', '2', '6']
2 -> ['3', '1', '5', '6']
6 -> ['1', '2', '7']
3 -> ['1', '2', '4']
5 -> ['2', '4']
4 -> ['3', '5']
7 -> ['6']
8 -> ['10', '11', '9']
9 -> ['10', '8']
10 -> ['8', '9']
11 -> ['8']
## Percurso em Largura
Vertice inicial:  1
Vertice visitado: 3; Predecessor : 1
Vertice visitado: 2; Predecessor : 1
Vertice visitado: 6; Predecessor : 1
Vertice visitado: 4; Predecessor : 3
Vertice visitado: 5; Predecessor : 2
Vertice visitado: 7; Predecessor : 6
Vertice visitado: 10; Predecessor : X
Vertice visitado: 8; Predecessor : 10
Vertice visitado: 9; Predecessor : 10
Vertice visitado: 11; Predecessor : 8


## Item **12** (Opcional) - Determinar se um grafo é bipartido
Para implementar essa função, foi usada a estratégia extraída da seção *Testing bipartiteness* no endereço *https://en.wikipedia.org/wiki/Bipartite_graph*.
O procedimento consiste em realizar um percurso em profundidade colorindo o grafo em duas cores - na implementação, as cores são representadas por 1 e -1. Caso seja possível colorir cada vértice com a cor oposta do seu vértice pai na árvore, o grafo é bipartido e a função retorna True, caso contrário, não é bipartido e retornamos False.

In [ ]:
def eh_bipartido(grafo):
    # Pega o vertice inicial
    vertice_inicial = list(grafo.get_vertices())[0]
    pilha = list()
    # Map para armazenar a cor de cada grafo
    vertices_coloridos = dict()
    # AS aresta serao representadas por 2-uplas
    arestas_visitadas = set()
    arestas_de_retorno = set()
    # Set para monitorar quais vertices ainda nao foram visitados
    vertices_nao_visitados = set(grafo.get_vertices())
    # Set para monitorar quais vertices ainda nao foram explorados
    vertices_nao_explorados = set(grafo.get_vertices())
    # Empilha o vértice inicial
    pilha.append(vertice_inicial)
    vertices_nao_visitados.remove(vertice_inicial)
    vertices_coloridos[vertice_inicial.label] = 1
    vizinhos_vertices = dict()
    # TODO : Essa variável é utilizada para armazenar a cor do último vértice visitado, para que em casos de grafos não
    # conexos, a nova raíz tenha a cor oposta. Não sei se isso funciona ou se é necessário.
    ultima_cor = 1 # Usado para grafos nao conexos
    while vertices_nao_explorados:
        if len(pilha) == 0:
            menor_label = min([i.label for i in vertices_nao_visitados])
            nova_raiz = grafo.get_vertice(min([i.label for i in vertices_nao_visitados]))
            pilha.append(nova_raiz)
            vertices_nao_visitados.remove(nova_raiz)
            # Colore o grafo com a cor oposta à última cor
            vertices_coloridos[nova_raiz.label] = -ultima_cor
        u = pilha[-1]
        if u not in vizinhos_vertices:
                labels_ordenadas = sorted([i.label for i in grafo.get_adjacentes(u.label)], reverse=True)
                vizinhos_vertices[u] = [grafo.get_vertice(v) for v in labels_ordenadas]

        v = vizinhos_vertices[u].pop() if len(vizinhos_vertices[u]) > 0 else None
        if not v:
            pilha.pop()
            vertices_nao_explorados.remove(u) # O vertice eh marcado como explorado
        elif v in vertices_nao_visitados:
            # Se não foi visitado, pinta com a cor oposta ao pai
            vertices_coloridos[v.label] = -vertices_coloridos[u.label]
            vertices_nao_visitados.remove(v)
            pilha.append(v)
            arestas_visitadas.add((u,v))
        else:
            if v in vertices_nao_explorados and pilha[-2] != v: # Checa se é adjaccente na pilha
                if (v,u) not in arestas_visitadas:
                    arestas_visitadas.add((u,v))
                    arestas_de_retorno.add((u,v))
                    # Se ja foi visitado e a sua cor não é compatível com a do vértice que se conecta a ele,
                    #  não é bipartido
                    if vertices_coloridos[u.label] != -vertices_coloridos[v.label] :
                        return False
    return True # Se foi possível fazer toda a coloração, o grafo é bipartido


In [ ]:
grafo = MatrizAdjacencia("Grafo 1", "./data/GRAFO_1.txt")
print(f"O grafo eh bipartido: {eh_bipartido(grafo)}")

O grafo eh bipartido: True


## Item **20** - Busca em profundidade, com determinação de profundidade de entrada e de saída de cada vértice, e arestas de árvore, retorno, avanço e cruzamento.

Para implementar essa função, foi usado o algoritmo passado em sala de aula que está presente nos slides.
O procedimento consiste em ler os arquivos que possuem os dados de cada dígrafo, criar a lista de adjacência e implementar a busca em profundidade em dígrafos classificando as arestas, baseado no algoritmo dos slides.

In [16]:
"""
  Lê um arquivo com os dados dos dígrafos.
  Retorna uma lista de adjacência {u: [v1, v2, ...]}.
"""
def ler_digrafo(caminho):
    adj = {}
    with open(caminho, 'r', encoding='utf-8') as f:
        linhas = [l.strip() for l in f.readlines() if l.strip()]
        _ = linhas[0]
        for linha in linhas[1:]:
            u, v = linha.split(',')
            u, v = u.strip(), v.strip()
            if u not in adj:
                adj[u] = []
            adj[u].append(v)
            if v not in adj:
                adj[v] = []
    return adj

"""
  Executa busca em profundidade em dígrafo e classifica arestas.
  Entrada: Lista de Adjacência {u: [v1, v2, ...]}
  Saída:
    - descoberta: Dicionário que mapeia o vértice ao tempo que foi descoberto;
    - termino: Dicionário que mapeia o vértice ao tempo que terminou;
    - pai: Mapeia cada vértice ao seu pai;
    - tipo_aresta: Mapeia cada aresta (u, v) ao seu tipo
"""
def dfs_digrafo(adj):
    # WHITE: Não processado, GRAY: Em processamento, BLACK: Processado
    WHITE, GRAY, BLACK = 0, 1, 2
    cor = {v: WHITE for v in adj}
    tempo = 0
    descoberta = {}
    termino = {}
    pai = {v: None for v in adj}
    tipo_aresta = {}

  #  Percorrer profundamente todos os vértices alcançáveis a partir de u,
  #  atualizando:
  #  os tempos de descoberta e término,
  #  a cor de cada vértice,
  #  e classificando as arestas.
    def dfs_visit(u):
        nonlocal tempo
        cor[u] = GRAY
        tempo += 1
        descoberta[u] = tempo
        for v in adj[u]:
            if cor[v] == WHITE:
                tipo_aresta[(u, v)] = "árvore"
                pai[v] = u
                dfs_visit(v)
            elif cor[v] == GRAY:
                tipo_aresta[(u, v)] = "retorno"
            else:  # cor[v] == BLACK
                if descoberta[u] < descoberta[v]:
                    tipo_aresta[(u, v)] = "avanço"
                else:
                    tipo_aresta[(u, v)] = "cruzamento"
        cor[u] = BLACK
        tempo += 1
        termino[u] = tempo

    for v in adj:
        if cor[v] == WHITE:
            dfs_visit(v)

    return descoberta, termino, pai, tipo_aresta

"""
  Executa busca em profundidade no dígrafo de um arquivo informado.
"""
def executar_dfs_em_arquivo(caminho):
    print(f"\n=== {caminho} ===")
    adj = ler_digrafo(caminho)
    disc, fini, pai, tipo = dfs_digrafo(adj)

    print("\nTempos de descoberta e término:")
    for v in sorted(disc, key=lambda x: disc[x]):
        print(f"{v}: d={disc[v]}, f={fini[v]}")

    print("\nArestas classificadas:")
    for (u, v), t in tipo.items():
        print(f"({u} -> {v}): {t}")

# ---- Executar nos arquivos fornecidos ----
arquivos = [
    "DIGRAFO_0.txt",
    "DIGRAFO1.txt",
    "DIGRAFO2.txt",
    "DIGRAFO3.txt",
]

for arq in arquivos:
    try:
        executar_dfs_em_arquivo("data/" + arq)
    except FileNotFoundError:
        print(f"Arquivo não encontrado.")



=== data/DIGRAFO_0.txt ===

Tempos de descoberta e término:
a: d=1, f=16
b: d=2, f=13
c: d=3, f=10
f: d=4, f=9
g: d=5, f=8
h: d=6, f=7
e: d=11, f=12
d: d=14, f=15

Arestas classificadas:
(a -> b): árvore
(b -> c): árvore
(c -> a): retorno
(c -> f): árvore
(f -> g): árvore
(g -> h): árvore
(b -> e): árvore
(e -> f): cruzamento
(a -> d): árvore
(d -> e): cruzamento

=== data/DIGRAFO1.txt ===

Tempos de descoberta e término:
1: d=1, f=20
2: d=2, f=19
3: d=3, f=18
4: d=4, f=17
5: d=5, f=16
6: d=6, f=15
7: d=7, f=14
9: d=8, f=13
8: d=9, f=12
10: d=10, f=11
11: d=21, f=26
12: d=22, f=25
13: d=23, f=24

Arestas classificadas:
(1 -> 2): árvore
(2 -> 3): árvore
(3 -> 1): retorno
(3 -> 4): árvore
(4 -> 5): árvore
(5 -> 6): árvore
(6 -> 7): árvore
(7 -> 6): retorno
(7 -> 9): árvore
(9 -> 8): árvore
(8 -> 4): retorno
(8 -> 10): árvore
(5 -> 8): avanço
(11 -> 12): árvore
(12 -> 13): árvore
(13 -> 12): retorno

=== data/DIGRAFO2.txt ===

Tempos de descoberta e término:
1: d=1, f=24
2: d=2, f=23
3: 